# LAPTOP

In this project, we'll learn the basics of data cleaning with pandas as we work with laptops.csv, a CSV file containing information about 1,300 laptop computers.

**1. Reading CSV Files with Encodings**

In [1]:
import pandas as pd
import numpy as np
laptops=pd.read_csv('laptops.csv', encoding='Latin-1')
laptops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 13 columns):
Manufacturer                1303 non-null object
Model Name                  1303 non-null object
Category                    1303 non-null object
Screen Size                 1303 non-null object
Screen                      1303 non-null object
CPU                         1303 non-null object
RAM                         1303 non-null object
 Storage                    1303 non-null object
GPU                         1303 non-null object
Operating System            1303 non-null object
Operating System Version    1133 non-null object
Weight                      1303 non-null object
Price (Euros)               1303 non-null object
dtypes: object(13)
memory usage: 132.4+ KB


The column labels have a variety of upper and lowercase letters, as well as spaces and parentheses, which will make them harder to work with and read. One noticeable issue is that the " Storage" column name has a space in front of it. These quirks with column labels can sometimes be hard to spot, so removing extra whitespaces from all column names will save us more work in the long run.

**2. Cleaning column names**

In [2]:
def convert(string):
    string=string.strip()
    string=string.replace('Operating System','os')
    string=string.replace(' ','_')
    string=string.replace('(','')
    string=string.replace(')','')
    string=string.lower()
    return string

new_col=[]
for i in laptops.columns:
    i=convert(i)
    new_col.append(i)
laptops.columns=new_col
print(laptops.columns)
laptops.head(1)

Index(['manufacturer', 'model_name', 'category', 'screen_size', 'screen',
       'cpu', 'ram', 'storage', 'gpu', 'os', 'os_version', 'weight',
       'price_euros'],
      dtype='object')


,manufacturer,model_name,category,screen_size,screen,cpu,ram,storage,gpu,os,os_version,weight,price_euros
0,Apple,MacBook Pro,Ultrabook,"13.3""",IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,NaN,1.37kg,"1339,69"


**4. Converting String Columns to Numeric**

We observed earlier that all 13 columns have the object dtype, meaning they're stored as strings.
Because the values in the screen_size column are stored as text data, we can't sort them numerically. For instance, if we wanted to select laptops with screens 15" or larger, we'd be unable to do so. 


In [3]:
#unique value of laptops' screen size
unique_screen=laptops['screen_size'].unique()
print(unique_screen)

['13.3"' '15.6"' '15.4"' '14.0"' '12.0"' '11.6"' '17.3"' '10.1"' '13.5"'
 '12.5"' '13.0"' '18.4"' '13.9"' '12.3"' '17.0"' '15.0"' '14.1"' '11.3"']


We can observe the following:

*    All values in this column follow the same pattern - a series of digit and period characters, followed by a quote character (").
*    There are no special cases. Every value matches the same pattern.
*    We'll need to convert the column to a float dtype, as the int dtype won't be able to store the decimal values.


In [4]:
laptops['screen_size']=laptops['screen_size'].str.replace('"','').astype(float)
print(laptops['screen_size'].unique())

laptops.rename({"screen_size": "screen_size_inches"}, axis=1, inplace=True)

[13.3 15.6 15.4 14.  12.  11.6 17.3 10.1 13.5 12.5 13.  18.4 13.9 12.3
 17.  15.  14.1 11.3]


Similarly, we have to remove non-digit characters in ram column and convert them to numeric dtypes next. 

In [5]:
unique_ram=laptops['ram'].unique()
print(unique_ram)

['8GB' '16GB' '4GB' '2GB' '12GB' '6GB' '32GB' '24GB' '64GB']


Because thee GB characters contained useful information about the units (gigabytes) of the laptop's ram, we rename the column from ram to ram_gb

In [6]:
laptops['ram']=laptops['ram'].str.replace('GB','').astype(int)
laptops.rename({"ram": "ram_gb"}, axis=1, inplace=True)

Values in weight column are under string type. We can convert to numeric values and rename column to weight_kg.

In [7]:
laptops["weight"] = laptops["weight"].str.replace("kgs","")
laptops['weight']=laptops['weight'].str.replace("kg","")
laptops['weight']=laptops['weight'].astype(float)
laptops.rename({"weight": "weight_kg"}, axis=1, inplace=True)

Similarly, we need to clean price_euros column, extract screen solution from screen column, and extract the processor speed from te cpu column

In [8]:
#Convert the price_euros column to a numeric dtype
laptops['price_euros']=laptops['price_euros'].str.replace(",",".").astype(float)

#Extract the screen resolution from the screen column.
laptops['screen_resolution']=laptops['screen'].str.split().str[-1]

#Extract the processor speed from the cpu column.
laptops['processor_speed']=laptops['cpu'].str.split().str[-1]

laptops.head()

,manufacturer,model_name,category,screen_size_inches,screen,cpu,ram_gb,storage,gpu,os,os_version,weight_kg,price_euros,screen_resolution,processor_speed
0,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8,128GB SSD,Intel Iris Plus Graphics 640,macOS,NaN,1.37,1339.69,2560x1600,2.3GHz
1,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8,128GB Flash Storage,Intel HD Graphics 6000,macOS,NaN,1.34,898.94,1440x900,1.8GHz
2,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8,256GB SSD,Intel HD Graphics 620,No OS,NaN,1.86,575.00,1920x1080,2.5GHz
3,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16,512GB SSD,AMD Radeon Pro 455,macOS,NaN,1.83,2537.45,2880x1800,2.7GHz
4,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8,256GB SSD,Intel Iris Plus Graphics 650,macOS,NaN,1.37,1803.60,2560x1600,3.1GHz


**5. Extracting Values from Strings**

Sometimes, it can be useful to extract non-numeric values from within strings.

In [9]:
laptops['gpu'].head()

0    Intel Iris Plus Graphics 640
1          Intel HD Graphics 6000
2           Intel HD Graphics 620
3              AMD Radeon Pro 455
4    Intel Iris Plus Graphics 650
Name: gpu, dtype: object

The information in this column seems to be a manufacturer (Intel, AMD) followed by a model name/number.

In [10]:
laptops["gpu_manufacturer"] = (laptops["gpu"].str.split().str[0])
laptops['cpu_manufacturer']=laptops["cpu"].str.split().str[0]
laptops.head()

,manufacturer,model_name,category,screen_size_inches,screen,cpu,ram_gb,storage,gpu,os,os_version,weight_kg,price_euros,screen_resolution,processor_speed,gpu_manufacturer,cpu_manufacturer
0,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8,128GB SSD,Intel Iris Plus Graphics 640,macOS,NaN,1.37,1339.69,2560x1600,2.3GHz,Intel,Intel
1,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8,128GB Flash Storage,Intel HD Graphics 6000,macOS,NaN,1.34,898.94,1440x900,1.8GHz,Intel,Intel
2,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8,256GB SSD,Intel HD Graphics 620,No OS,NaN,1.86,575.00,1920x1080,2.5GHz,Intel,Intel
3,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16,512GB SSD,AMD Radeon Pro 455,macOS,NaN,1.83,2537.45,2880x1800,2.7GHz,AMD,Intel
4,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8,256GB SSD,Intel Iris Plus Graphics 650,macOS,NaN,1.37,1803.60,2560x1600,3.1GHz,Intel,Intel


Counts of each manufacturer in gpu_manufacture and cpu_manufacturer

In [11]:
print(laptops['gpu_manufacturer'].value_counts())

print(laptops['cpu_manufacturer'].value_counts())

Intel     722
Nvidia    400
AMD       180
ARM         1
Name: gpu_manufacturer, dtype: int64
Intel      1240
AMD          62
Samsung       1
Name: cpu_manufacturer, dtype: int64


**6. Correcting bad values**

If your data has been scraped from a webpage or if there was manual data entry involved at some point, you may end up with inconsistent values. 

In [12]:
print(laptops["os"].value_counts())

Windows      1125
No OS          66
Linux          62
Chrome OS      27
macOS          13
Mac OS          8
Android         2
Name: os, dtype: int64


There are two variations of the Apple operating system in our data set: Mac OS and macOS. We create a dictionary to use with mapping to correct the values in the os column.

In [13]:
mapping_dict = {
    'Android': 'Android',
    'Chrome OS': 'Chrome OS',
    'Linux': 'Linux',
    'Mac OS': 'macOS',
    'No OS': 'No OS',
    'Windows': 'Windows',
    'macOS': 'macOS'
}
laptops['os']=laptops['os'].map(mapping_dict)

In [14]:
print(laptops["os"].value_counts())

Windows      1125
No OS          66
Linux          62
Chrome OS      27
macOS          21
Android         2
Name: os, dtype: int64


**7. Filling missing values**


Count the null values for each column

In [15]:
print(laptops.isnull().sum())

manufacturer            0
model_name              0
category                0
screen_size_inches      0
screen                  0
cpu                     0
ram_gb                  0
storage                 0
gpu                     0
os                      0
os_version            170
weight_kg               0
price_euros             0
screen_resolution       0
processor_speed         0
gpu_manufacturer        0
cpu_manufacturer        0
dtype: int64


There are 170 missing values in the os_version column. We can remove or drop rows and columns with null values but it may remove an inappropriate amount of data which could change our analysis. We need to explore the missing values in the os_version first.

In [16]:
print(laptops['os_version'].value_counts(dropna=False))

10      1072
NaN      170
7         45
X          8
10 S       8
Name: os_version, dtype: int64


Most of values in the column are 10 and the next most common value is NaN. Let's also explore the os column, since it's is closely related to the os_version column. We'll only look at rows in which the os_version is missing:

In [17]:
value_counts_before = (laptops.loc[laptops["os_version"]
                                   .isnull(), "os"].value_counts())
print(value_counts_before)

No OS        66
Linux        62
Chrome OS    27
macOS        13
Android       2
Name: os, dtype: int64


Most of the value are "No OS", we can replace them with "Version Unknown".
13 of the laptops having macOS so the os_version should be "X".
Thus, we can fill the missing values to make our data more correct. For the rest of the values, it is better to leave them as missing than being removed.


In [18]:
#Filling os_version of laptops with macOS value in os column
laptops.loc[laptops["os"] == "macOS", "os_version"] = "X"

#Correcting "NoOS" value to "Version Unknown"
laptops.loc[laptops['os']=='No OS','os_version']='Version Unknown'

#View values in os column that have missing value in os_version column again
value_counts_after = (laptops.loc[laptops["os_version"]
                                  .isnull(), "os"]
                      .value_counts())
print(value_counts_after)


Linux        62
Chrome OS    27
Android       2
Name: os, dtype: int64


**8. Some analysis**

**Are laptops made by Apple more expensive than those made by other manufacturers?**

In [19]:
#Average price of laptops maded by Apple
laptops_Apple=laptops.loc[laptops['manufacturer']=='Apple', 'price_euros'].mean()

#Average price of other laptops
laptops_Others=laptops.loc[laptops['manufacturer']!='Apple', 'price_euros'].mean()

print(f'Average price of laptops maded by Apple is {laptops_Apple:.2f} euro')
print(f'Average price of laptops maded by other manufacturer is {laptops_Others:.2f} euro')


Average price of laptops maded by Apple is 1564.20 euro
Average price of laptops maded by other manufacturer is 1116.47 euro


In [20]:
print('Are laptops made by Apple more expensive than those made by other manufacturers?')
laptops_Apple > laptops_Others

Are laptops made by Apple more expensive than those made by other manufacturers?


True

**What is the best value laptop with a screen size of 15" or more?**

In [21]:
laptops_from_15inch=laptops[laptops['screen_size_inches']>=15]

#Laptop from 15 inch having highest price
laptops_15inch_best=(laptops_from_15inch[laptops_from_15inch['price_euros']==laptops_from_15inch['price_euros']
                                         .max(axis=0)])
print('The best value laptop with a screen size of 15" or more is')
laptops_15inch_best

The best value laptop with a screen size of 15" or more is


,manufacturer,model_name,category,screen_size_inches,screen,cpu,ram_gb,storage,gpu,os,os_version,weight_kg,price_euros,screen_resolution,processor_speed,gpu_manufacturer,cpu_manufacturer
196,Razer,Blade Pro,Gaming,17.3,4K Ultra HD / Touchscreen 3840x2160,Intel Core i7 7820HK 2.9GHz,32,1TB SSD,Nvidia GeForce GTX 1080,Windows,10,3.49,6099.0,3840x2160,2.9GHz,Nvidia,Intel


**Which laptop has the most storage space?**


In [22]:
laptops['storage'].unique()

array(['128GB SSD', '128GB Flash Storage', '256GB SSD', '512GB SSD',
       '500GB HDD', '256GB Flash Storage', '1TB HDD',
       '32GB Flash Storage', '128GB SSD +  1TB HDD',
       '256GB SSD +  256GB SSD', '64GB Flash Storage',
       '256GB SSD +  1TB HDD', '256GB SSD +  2TB HDD', '32GB SSD',
       '2TB HDD', '64GB SSD', '1TB Hybrid', '512GB SSD +  1TB HDD',
       '1TB SSD', '256GB SSD +  500GB HDD', '128GB SSD +  2TB HDD',
       '512GB SSD +  512GB SSD', '16GB SSD', '16GB Flash Storage',
       '512GB SSD +  256GB SSD', '512GB SSD +  2TB HDD',
       '64GB Flash Storage +  1TB HDD', '1GB SSD', '1TB HDD +  1TB HDD',
       '32GB HDD', '1TB SSD +  1TB HDD', '512GB Flash Storage',
       '128GB HDD', '240GB SSD', '8GB SSD', '508GB Hybrid',
       '512GB SSD +  1TB Hybrid', '256GB SSD +  1TB Hybrid'], dtype=object)

Each laptop has 1 or 2 types of storage. We need to clean this column first:
* Split values in storage column 
* Select the first value and/or the second last value in splited result 
* Convert to numerical type
* Convert to the same measure of GB
* Created new column with name storage_gb


In [23]:
#Split storage column
laptops['storage'].str.split()

0                      [128GB, SSD]
1           [128GB, Flash, Storage]
2                      [256GB, SSD]
3                      [512GB, SSD]
4                      [256GB, SSD]
5                      [500GB, HDD]
6           [256GB, Flash, Storage]
7           [256GB, Flash, Storage]
8                      [512GB, SSD]
9                      [256GB, SSD]
10                     [500GB, HDD]
11                     [500GB, HDD]
12                     [256GB, SSD]
13                     [256GB, SSD]
14                     [256GB, SSD]
15                     [256GB, SSD]
16                     [256GB, SSD]
17                     [512GB, SSD]
18                       [1TB, HDD]
19                     [128GB, SSD]
20           [32GB, Flash, Storage]
21        [128GB, SSD, +, 1TB, HDD]
22                     [500GB, HDD]
23                     [256GB, SSD]
24                     [256GB, SSD]
25                       [1TB, HDD]
26          [128GB, Flash, Storage]
27                     [256G

Extract numerical information from the splitted result

In [24]:
laptops['storage_1']=laptops['storage'].str.split().str[0]
laptops['storage_2']=laptops['storage'].str.split().str[-2]
laptops.head()

,manufacturer,model_name,category,screen_size_inches,screen,cpu,ram_gb,storage,gpu,os,os_version,weight_kg,price_euros,screen_resolution,processor_speed,gpu_manufacturer,cpu_manufacturer,storage_1,storage_2
0,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8,128GB SSD,Intel Iris Plus Graphics 640,macOS,X,1.37,1339.69,2560x1600,2.3GHz,Intel,Intel,128GB,128GB
1,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8,128GB Flash Storage,Intel HD Graphics 6000,macOS,X,1.34,898.94,1440x900,1.8GHz,Intel,Intel,128GB,Flash
2,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8,256GB SSD,Intel HD Graphics 620,No OS,Version Unknown,1.86,575.00,1920x1080,2.5GHz,Intel,Intel,256GB,256GB
3,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16,512GB SSD,AMD Radeon Pro 455,macOS,X,1.83,2537.45,2880x1800,2.7GHz,AMD,Intel,512GB,512GB
4,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8,256GB SSD,Intel Iris Plus Graphics 650,macOS,X,1.37,1803.60,2560x1600,3.1GHz,Intel,Intel,256GB,256GB


The information of first storage of laptops is easily to convert.

In [25]:
laptops['storage_1']=laptops['storage_1'].str.replace('GB','').str.replace('TB','000').astype(int)
laptops.head(1)

,manufacturer,model_name,category,screen_size_inches,screen,cpu,ram_gb,storage,gpu,os,os_version,weight_kg,price_euros,screen_resolution,processor_speed,gpu_manufacturer,cpu_manufacturer,storage_1,storage_2
0,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8,128GB SSD,Intel Iris Plus Graphics 640,macOS,X,1.37,1339.69,2560x1600,2.3GHz,Intel,Intel,128,128GB


Laptops may not have second storage so we just replace those value with 0

In [26]:
laptops['storage_2']=laptops['storage_2'].str.replace('Flash','0')
laptops['storage_2']=laptops['storage_2'].str.replace('GB','').str.replace('TB','000').astype(int)

laptops.head()

,manufacturer,model_name,category,screen_size_inches,screen,cpu,ram_gb,storage,gpu,os,os_version,weight_kg,price_euros,screen_resolution,processor_speed,gpu_manufacturer,cpu_manufacturer,storage_1,storage_2
0,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8,128GB SSD,Intel Iris Plus Graphics 640,macOS,X,1.37,1339.69,2560x1600,2.3GHz,Intel,Intel,128,128
1,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8,128GB Flash Storage,Intel HD Graphics 6000,macOS,X,1.34,898.94,1440x900,1.8GHz,Intel,Intel,128,0
2,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8,256GB SSD,Intel HD Graphics 620,No OS,Version Unknown,1.86,575.00,1920x1080,2.5GHz,Intel,Intel,256,256
3,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16,512GB SSD,AMD Radeon Pro 455,macOS,X,1.83,2537.45,2880x1800,2.7GHz,AMD,Intel,512,512
4,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8,256GB SSD,Intel Iris Plus Graphics 650,macOS,X,1.37,1803.60,2560x1600,3.1GHz,Intel,Intel,256,256


Sum of storage 1 and 2 is the total storage of laptop

In [27]:
laptops['storage_gb']=laptops['storage_1']+laptops['storage_2']
laptops.head()

,manufacturer,model_name,category,screen_size_inches,screen,cpu,ram_gb,storage,gpu,os,os_version,weight_kg,price_euros,screen_resolution,processor_speed,gpu_manufacturer,cpu_manufacturer,storage_1,storage_2,storage_gb
0,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8,128GB SSD,Intel Iris Plus Graphics 640,macOS,X,1.37,1339.69,2560x1600,2.3GHz,Intel,Intel,128,128,256
1,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8,128GB Flash Storage,Intel HD Graphics 6000,macOS,X,1.34,898.94,1440x900,1.8GHz,Intel,Intel,128,0,128
2,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8,256GB SSD,Intel HD Graphics 620,No OS,Version Unknown,1.86,575.00,1920x1080,2.5GHz,Intel,Intel,256,256,512
3,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16,512GB SSD,AMD Radeon Pro 455,macOS,X,1.83,2537.45,2880x1800,2.7GHz,AMD,Intel,512,512,1024
4,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8,256GB SSD,Intel Iris Plus Graphics 650,macOS,X,1.37,1803.60,2560x1600,3.1GHz,Intel,Intel,256,256,512


The biggest storage (GB) that laptops in data set could achieve is

In [28]:
top_storage=laptops['storage_gb'].max(axis=0)
print(top_storage)

4000


Laptops with the most storage space are

In [29]:
laptops_top_storage=laptops[laptops['storage_gb']==4000]
laptops_top_storage['manufacturer'].value_counts()

HP        7
Lenovo    5
Dell      3
Asus      1
Name: manufacturer, dtype: int64